Principales transformaciones y acciones
======================================
### Transformaciones elemento-a-elemento
Generan un nuevo RDD a partir de uno dado

-   `filter(func)` filtra los elementos de un RDD

In [1]:
!pip install test_helper

In [2]:
from test_helper import Test
from __future__ import print_function


rdd = sc.parallelize(xrange(-5,5))          # Rango (-5, 5)
filtered_rdd = rdd.filter(lambda x: x>=0)   # Devuelve los positivos
print(filtered_rdd.collect())
Test.assertEquals(filtered_rdd.collect(), [0, 1, 2, 3, 4])

[0, 1, 2, 3, 4]
1 test passed.


-   `map(func)` aplica una función a los elementos de un RDD

In [3]:
def add1(x):
    return(x+1)

squared_rdd = (filtered_rdd
               .map(add1)
               .map(lambda x: (x, x*x))) 
print(squared_rdd.collect())
Test.assertEquals(squared_rdd.collect(), [(1, 1), (2, 4), (3, 9), (4, 16), (5, 25)])

[(1, 1), (2, 4), (3, 9), (4, 16), (5, 25)]
1 test passed.


-   `flatMap(func)` igual que `map`, pero “aplana” la salida

In [4]:
squaredflat_rdd = (filtered_rdd
                   .map(add1)
                   .flatMap(lambda x: (x, x*x)))
print(squaredflat_rdd.collect())
Test.assertEquals(squaredflat_rdd.collect(), [1, 1, 2, 4, 3, 9, 4, 16, 5, 25])

[1, 1, 2, 4, 3, 9, 4, 16, 5, 25]
1 test passed.


-   `sample(withReplacement, fraction, seed=None)` devuelve una muestra del RDD
    - `withReplacement` - si True, cada elemento puede aparecer varias veces en la muestra
    - `fraction` - tamaño esperado de la muestra como una fracción del tamaño del RDD (sin reemplazo: probabilidad de seleccionar un elemento, su valor debe ser [0, 1]; con reemplazo: número esperado de veces que que se escoge un elemento, su valor debe ser >= 0
    - `seed` - semilla para el generador de números aleatorios

In [5]:
s1 = squaredflat_rdd.sample(False, 0.5).collect()
s2 = squaredflat_rdd.sample(True, 2).collect()
s3 = squaredflat_rdd.sample(False, 0.8).collect()
print('s1={0}\ns2={1}\ns3={2}'.format(s1, s2, s3))

s1=[1, 2, 4, 3, 9, 4, 5, 25]
s2=[1, 1, 1, 4, 4, 3, 3, 3, 9, 9, 9, 9, 16, 16, 5, 25, 25]
s3=[1, 1, 2, 4, 3, 9, 4, 5]


-   `distinct()` devuelve un nuevo RDD sin duplicados

In [6]:
distinct_rdd = squaredflat_rdd.distinct()
print(distinct_rdd.collect())

[16, 4, 1, 25, 5, 9, 2, 3]


-   `groupBy(func)` devuelve un RDD con los datos agrupados en formato
    clave/valor, usando una función para obtener la clave

In [7]:
grouped_rdd = distinct_rdd.groupBy(lambda x: x%3)
print(grouped_rdd.collect())
print([(x,sorted(y)) for (x,y) in grouped_rdd.collect()])

[(0, <pyspark.resultiterable.ResultIterable object at 0x7fa3997653d0>), (1, <pyspark.resultiterable.ResultIterable object at 0x7fa3990bbe10>), (2, <pyspark.resultiterable.ResultIterable object at 0x7fa3990d1090>)]
[(0, [3, 9]), (1, [1, 4, 16, 25]), (2, [2, 5])]


### Transformaciones sobre dos RDDs

Operaciones tipo conjunto sobre dos RDDs

-   `union(rdd)` produce un RDD con los datos de los dos de partida

In [8]:
rdda = sc.parallelize(['a', 'b', 'c'])
rddb = sc.parallelize(['c', 'd', 'e'])
rddu = rdda.union(rddb)
Test.assertEquals(rddu.collect(),['a', 'b', 'c', 'c', 'd', 'e'])

1 test passed.


-   `intersection(rdd)` elementos en ambos RDDs

In [9]:
rddi = rdda.intersection(rddb)
Test.assertEquals(rddi.collect(),['c'])

1 test passed.


-   `subtract(rdd)` los datos del primer RDD menos los del segundo

In [10]:
rdds = rdda.subtract(rddb)
Test.assertEquals(rdds.collect(), ['a', 'b'])

1 test passed.


-   `cartesian(rdd)` producto cartesiano de ambos RDDs (operación muy
    costosa)

In [11]:
rddc = rdda.cartesian(rddb)
Test.assertEquals(rddc.collect(), 
                  [('a','c'),('a','d'),('a','e'),('b','c'),('b','d'),('b','e'),('c','c'), ('c','d'), ('c','e')])

1 test passed.


### Acciones sobre RDDs simples

#### Acciones de agregación

-   `reduce(func)` combina los elementos del RDD en paralelo
    - La función de reducción debe ser un monoide conmutativo
    - Primero se realiza la redución a nivel de partición

In [12]:
rdd = sc.parallelize(xrange(1,5))
r = rdd.reduce(lambda x,y: x*y) # r = 1*2*3*4 = 24
from operator import add
s = rdd.reduce(add) # s = 1+2+3+4 = 10
Test.assertEquals(r, 24)
Test.assertEquals(s, 10)

1 test passed.
1 test passed.


-   `fold(cero, func)` similar a `reduce`, pero proporcionando el valor
    identidad para la función (p.e. 0 para $+$; 1 para $\times$, o `[]`
    para concatenación de listas)
    - `cero` se usa como valor inicial para cada partición
    - `func` debe ser un monoide conmutativo para garantizar un resultado consistente

In [13]:
rdd = sc.parallelize([range(1,5), range(-10,-3), ['a', 'b', 'c']], 2)
print(rdd.glom().collect())
f = rdd.fold([], lambda x,y: x+y)
print(f)
Test.assertEquals(f, [1, 2, 3, 4, -10, -9, -8, -7, -6, -5, -4, 'a', 'b', 'c'])

[[[1, 2, 3, 4]], [[-10, -9, -8, -7, -6, -5, -4], ['a', 'b', 'c']]]
[1, 2, 3, 4, -10, -9, -8, -7, -6, -5, -4, 'a', 'b', 'c']
1 test passed.


- `aggregate(cero,seqOp,combOp)` devuelve una colección agregando los elementos del RDD usando dos funciones:
    1. `seqOp` -  agregación a nivel de partición: se crea un acumulador por partición (inicializado a `cero`) y se agregan los valores de la partición en el acumulador
    2. `combOp` - agregación entre particiones: se agregan los acumuladores de todas las particiiones
    
 - Ambas agregaciones usan un valor inicial `cero` (similar al caso de `fold`).
 - Permite devolver un tipo diferente del de los elementos del RDD de entrada.

In [14]:
l = [1, 2, 3, 4, 5, 6, 7, 8]
rdd = sc.parallelize(l, 2)
seqOp  = (lambda acc, val: (acc[0]+[val], acc[1]*val, acc[2]+1))       
combOp = (lambda acc1, acc2: (acc1[0]+acc2[0], acc1[1]*acc2[1], acc1[2]+acc2[2]))
a = rdd.aggregate(([], 1., 0), seqOp, combOp) 
Test.assertEquals(a[0], l)
Test.assertEquals(a[1], 8.*7.*6.*5.*4.*3.*2.*1.)
Test.assertEquals(a[2], len(l))

1 test passed.
1 test passed.
1 test passed.


#### Acciones para contar elementos

- `count()` devuelve un entero con el número de elementos del RDD

- `countApprox(timeout, confidence=0.95)` versión aproximada de `count()` que devuelve un resultado potencialmente incompleto en un tiempo máximo, incluso si no todas las tareas han finalizado. (Experimental).
- `countApproxDistinct(relativeSD=0.05)` devuelve una estimación del número de elementos diferentes del RDD.  (Experimental).
    - `relativeSD` – exactitud relativa (valores más pequeños implican menor error, pero requieren más memoria; debe ser mayor que 0.000017).

In [15]:
rdd = sc.parallelize([i % 20 for i in range(1000)])
Test.assertEquals(rdd.count(), 1000)
Test.assertEquals(rdd.distinct().count(), 20)
Test.assertTrue(900 < rdd.countApprox(1, 0.4) < 1100)
Test.assertTrue(15 < rdd.countApproxDistinct(0.5) < 25)

1 test passed.
1 test passed.
1 test passed.
1 test passed.



-   `countByValue()` devuelve el número de apariciones de cada elemento
    del RDD como un diccionario {valor:número}

In [16]:
rdd = sc.parallelize(list("abracadabra")).cache()
dicc = rdd.countByValue()
Test.assertEquals(dicc, {'a': 5, 'c': 1, 'b': 2, 'r': 2, 'd': 1})

1 test passed.


#### Acciones para obtener valores

-   `collect()` devuelve una lista con todos los elementos del RDD
    - Usar con cuidado, si el RDD es muy grande

In [17]:
lista = rdd.collect()
Test.assertEquals(lista, ['a','b','r','a','c','a','d','a','b','r','a'])

1 test passed.


-   `take(n)` devuelve `n` elementos del RDD
-   `takeSample(withRep, n, [seed])` devuelve `n` elementos aleatorios
    del RDD

In [18]:
t = rdd.take(4)
Test.assertEquals(t, ['a','b','r','a'])
s = rdd.takeSample(False, 4)
print(s)

1 test passed.
['r', 'd', 'a', 'a']


-   `top(n)` devuelve una lista con los primeros `n` elementos del RDD ordenado en
    orden descendente
-   `takeOrdered(n,[orden])` devuelve una lista con los primeros `n` elementos del RDD en orden
    ascendente, o siguiendo el orden indicado en la función opcional

In [19]:
rdd = sc.parallelize([10, 1, 2, 9, 3, 4, 5, 6, 7]).cache()
t = rdd.top(4)
o = rdd.takeOrdered(4)
i = rdd.takeOrdered(4, lambda x: -x)
Test.assertEquals(t, [10, 9, 7, 6])
Test.assertEquals(o, [1, 2, 3, 4])
Test.assertEquals(i, t)

1 test passed.
1 test passed.
1 test passed.


## Práctica 1

Usando countByValue, implementa un WordCount simple

In [20]:
import re
quijote = sc.textFile("datos/quijote.txt")
palabras = quijote.flatMap(lambda line: line.split(" "))
#print(palabras.collect())
wc = palabras.countByValue()
#print(wc.collect())
print(wc['Quijote'])
print(wc['Sancho'])
print(wc['Rocinante'])

Test.assertEquals(wc['Quijote'], 894)
Test.assertEquals(wc['Sancho'], 950)
Test.assertEquals(wc['Rocinante'], 71)

894
950
71
1 test passed.
1 test passed.
1 test passed.
